# Find Connected Entities

Given a list of processes, how to find their parent process? How to find their openned network connections? How to find files read/written by them?

## What you will learn

0. What are relations Kestrel can resolve?
1. How to `FIND` child processes of processes?
2. How to `FIND` parent processes of processes?
3. How to `FIND` any connected processes of processes?
4. How to `FIND` network-traffic of processes?
5. Exercise: find processes which connected to `13.86.101.172`

### 0. What are relations Kestrel can resolve?

Kestrel uses the [FIND](https://kestrel.readthedocs.io/en/latest/language.html#find) command to provide a uniform relation expression across different data sources and different versions/dialects of STIX.

You can specify the relation like `CREATED`, `CREATED BY`, `ACCEPTED`, `ACCEPTED BY`, `OWNED`, `OWNED BY` to resolve between two variables. A relation is directional, and Kestrel uses _VERB_ and _VERB BY_ in `FIND` to indicate which direction the relation is.

<img src="https://kestrel.readthedocs.io/en/latest/_images/entityrelation.png" alt="supported entity relations" width="800" align="left"/>

### 1.  How to `FIND` child processes of processes?

After getting the scheduler process on the monitored Windows host, you can find the scheduled tasks, which are the child processes of the scheduler `svchost.exe`. This child process retrieval command using `FIND` is also a very common hunt step to get commands executed in a shell.

In [1]:
svchost = GET process
          FROM file:///tmp/lab101.json
          WHERE [process:name = 'svchost.exe']
          START t'2021-04-03T00:00:00Z' STOP t'2021-04-03T02:00:00Z'
          
scheduler = GET process
            FROM svchost
            WHERE [process:command_line = 'C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule']
            
# finding the child processes of `scheduler`
scheduled_tasks = FIND process CREATED BY scheduler

DISP scheduled_tasks ATTR name, pid, command_line

name,pid,command_line
CompatTelRunner.exe,10544,C:\Windows\system32\compattelrunner.exe -maintenance
DiskSnapshot.exe,7940,C:\Windows\system32\disksnapshot.exe -z
MpCmdRun.exe,5536,"""C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2102.4-0\MpCmdRun.exe"" -IdleTask -TaskName WdCacheMaintenance"
MpCmdRun.exe,9004,"""C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2102.4-0\MpCmdRun.exe"" -IdleTask -TaskName WdCleanup"
MpCmdRun.exe,13972,"""C:\ProgramData\Microsoft\Windows Defender\Platform\4.18.2102.4-0\MpCmdRun.exe"" -IdleTask -TaskName WdVerification"
amcet.exe,2452,c:\programdata\wii\amcet.exe --logPath C:\ProgramData\wii\amcet.log
amcet.exe,13824,c:\programdata\wii\amcet.exe --logPath C:\ProgramData\wii\amcet.log
dstokenclean.exe,9672,C:\Windows\system32\dstokenclean.exe
sc.exe,13580,C:\Windows\system32\sc.exe start w32time task_started
schtasks.exe,7440,C:\Windows\System32\schtasks.exe /CHANGE /TN AD\RII-Set_User_Last_Logon /ru users


### 2. How to `FIND` parent processes of processes?

Now try `CREATED` instead of `CREATED BY` to find parent processes:

In [2]:
scheduler2 = FIND process CREATED scheduled_tasks

# display scheduler2 and scheduler to confirm they are the same
DISP scheduler2 ATTR name, pid, command_line
DISP scheduler ATTR name, pid, command_line

name 
 pid 
 command_line 
 
 
 
 
 svchost.exe 
 2224 
 C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule 
 
 
 
 
 
 
 name 
 pid 
 command_line 
 
 
 
 
 svchost.exe 
 2224 
 C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 scheduler2 
 process 
 18 
 18 
 1124 
 1186 
 3262 
 1946 
 1102 
 1014 
 1168 
 1098 
 2016 
 2016 
 2336 
 2024 
 2340 
 2348 
 2348 
 
 
 *Number of related records cached.

### 3. How to `FIND` any connected processes of processes?

Sometimes you may want to find both parent and child processes of a variable. Use the relation `LINKED` to resolve all relations going out from a variable:

In [3]:
svchost_parent = FIND process CREATED svchost

svchost_child = FIND process CREATED BY svchost

svchost_all_connected_procs = FIND process LINKED svchost

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,user-account*,x-ecs-destination*,x-ecs-network*,x-ecs-process*,x-ecs-source*,x-ecs-user*,x-oca-asset*,x-oca-event*
svchost_parent,process,2,2,1619,1675,4789,2867,1601,1521,1673,1595,4536,4536,4958,4554,4967,4985,4985
svchost_child,process,22,22,1691,1787,4901,2923,1657,1521,1765,1651,4536,4536,5290,4554,5299,5317,5317
svchost_all_connected_procs,process,413,557,1693,1791,4905,2925,1659,1521,1378,1653,4536,4536,5298,4554,5307,5325,5325


### 4. How to `FIND` network-traffic of processes?

Beyond finding processes of processes, you can find other connected entities such as `network-traffic`, `user-account`, and `file` from processes. 

In [4]:
nt = FIND network-traffic CREATED BY svchost

DISP nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value,src_port,dst_ref.value,dst_port
10.171.5.141,62962,10.22.254.211,49669
10.184.147.141,62959,10.184.147.22,389
10.171.5.141,63061,10.22.254.211,389
10.184.147.141,62958,10.184.147.22,389
10.184.147.141,62956,10.184.147.22,389
10.171.5.141,62992,10.22.254.211,389
10.171.5.141,63022,10.22.254.211,389
10.184.147.141,62951,10.184.147.22,389
10.171.5.141,62961,10.22.254.211,135
10.184.147.141,62957,10.184.147.22,389


### 5. Exercise: find processes which connected to `13.86.101.172`

1. Use `GET` to retrieve all `ipv4-addr` entities with IP `13.86.101.172` from any log on the monitored host.
2. Use `FIND` to retrieve network-traffic with destination to the first variable.
3. Use `FIND` to retrieve processes which connected to the network-traffic.

Tip: look up the relation keyword in the figure at the beginning of this hunt book or in the [FIND documentation](https://kestrel.readthedocs.io/en/latest/language.html#find).